In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

login(token='')

model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1")



/Users/subhojit/workspace/saturn/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/subhojit/workspace/saturn/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.78s/it]


In [ ]:
device = "mps"
model = model.to(device)

In [5]:
text = """Who is Albert Kurt Godel ?"""
encodeds = tokenizer(text, return_tensors="pt", add_special_tokens=False)

In [6]:
model_inputs = encodeds.to(device)


In [7]:
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])